In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

Py-Goldsberry - Player-Level Box Score Data
===

This tutorial walks through using the py-goldsberry package to collect box score data at the player level.

To get started, we need to import py-goldsberry and we're going to go ahead import pandas so we can explore the data quickly once we have it collected.

In [2]:
from __future__ import division

import goldsberry
import pandas as pd
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
goldsberry.__version__

'0.8.0.1'

## Getting List of All Games

In [3]:
game_list = goldsberry.GameIDs()

In [4]:
df_games = pd.DataFrame(game_list.game_list())

In [5]:
df_games.head()

,AST,BLK,DREB,FG3A,FG3M,FG3_PCT,FGA,FGM,FG_PCT,FTA,FTM,FT_PCT,GAME_DATE,GAME_ID,MATCHUP,MIN,OREB,PF,PLUS_MINUS,PTS,REB,SEASON_ID,STL,TEAM_ABBREVIATION,TEAM_ID,TEAM_NAME,TOV,VIDEO_AVAILABLE,WL
0,28,4,45,29,8,0.276,124,56,0.452,46,27,0.587,2015-12-18,0021500391,DET @ CHI,340,19,35,3,147,64,22015,7,DET,1610612765,Detroit Pistons,11,1,W
1,35,4,34,25,13,0.520,88,53,0.602,16,11,0.688,2016-02-25,0021500855,GSW @ ORL,240,8,21,16,130,42,22015,9,GSW,1610612744,Golden State Warriors,21,1,W
2,30,2,36,23,9,0.391,87,53,0.609,34,27,0.794,2016-01-02,0021500496,SAC vs. PHX,240,10,23,23,142,46,22015,9,SAC,1610612758,Sacramento Kings,15,1,W
3,36,8,26,18,10,0.556,76,52,0.684,22,18,0.818,2016-03-05,0021500926,MIN vs. BKN,240,4,26,14,132,30,22015,8,MIN,1610612750,Minnesota Timberwolves,12,1,W
4,28,11,34,12,6,0.500,77,52,0.675,25,19,0.760,2016-03-01,0021500892,MIA vs. CHI,240,4,23,18,129,38,22015,5,MIA,1610612748,Miami Heat,15,1,W


In [6]:
team_cols = ['TEAM_ID','TEAM_NAME','TEAM_ABBREVIATION','SEASON_ID',
             'GAME_DATE','GAME_ID','MATCHUP','WL',
             'PTS','REB','STL','TOV','AST','BLK','DREB','FG3A','FG3M',
             'FG3_PCT','FGA','FGM','FG_PCT','FTA','FTM','FT_PCT','MIN',
             'OREB','PF','PLUS_MINUS','VIDEO_AVAILABLE']

df_games = df_games[team_cols]

## Getting Player Level Data

In [7]:
player_list = goldsberry.PlayerList()

In [8]:
player_list.SET_parameters(Season = '2014-15')

In [9]:
df_players = pd.DataFrame(player_list.players())

In [10]:
df_players.head()

,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,FROM_YEAR,GAMES_PLAYED_FLAG,PERSON_ID,PLAYERCODE,ROSTERSTATUS,TEAM_ABBREVIATION,TEAM_CITY,TEAM_CODE,TEAM_ID,TEAM_NAME,TO_YEAR
0,Quincy Acy,"Acy, Quincy",2012,Y,203112,quincy_acy,1,SAC,Sacramento,kings,1610612758,Kings,2015
1,Jordan Adams,"Adams, Jordan",2014,Y,203919,jordan_adams,1,MEM,Memphis,grizzlies,1610612763,Grizzlies,2015
2,Steven Adams,"Adams, Steven",2013,Y,203500,steven_adams,1,OKC,Oklahoma City,thunder,1610612760,Thunder,2015
3,Arron Afflalo,"Afflalo, Arron",2007,Y,201167,arron_afflalo,1,NYK,New York,knicks,1610612752,Knicks,2015
4,Alexis Ajinca,"Ajinca, Alexis",2008,Y,201582,alexis_ajinca,1,NOP,New Orleans,pelicans,1610612740,Pelicans,2015


## Getting game logs for the entire league

Now that we know how to get the game log data for a single person, we can combine that knowledge with information in the `df_players` to loop through the entire league and create a dataset of player-level game logs for the entire league.

To do this, we're going to iterate over the `PERSON_ID` column in our `df_players`. We're going to save the results of each iteration to an ever expanding list, `league_logs`. Once we're done with the loop, we're going to convert it to a dataframe. 

In [11]:
player_log = goldsberry.player.game_logs(203112)

In [12]:
player_log.SET_parameters(Season='2014-15')

In [13]:
league_logs = []
for _ , pid in df_players.PERSON_ID.iteritems():
    player_log.SET_parameters(PlayerID = pid)
    player_log.GET_raw_data()
    league_logs[0:0] = player_log.logs()

df_gamelogs = pd.DataFrame(league_logs)

Because we don't remember all of the Player's names by their ID, we're going to use our `df_players` data frame to append the appropriate name and team to the game log data. Finally, we're going to rearrange the columns because it doesn't make sense to keep them in alphabetical order. The table will be easier to understand if it's in some reasonable order given the nature of the data. 

In [14]:
df_gamelogs = pd.merge(df_gamelogs, df_players.loc[:,['DISPLAY_FIRST_LAST', 'PERSON_ID']], left_on = 'Player_ID', right_on='PERSON_ID')

In [15]:
col_order = ['Player_ID','DISPLAY_FIRST_LAST',
             'SEASON_ID','GAME_DATE','Game_ID','MATCHUP','WL',
             'PTS','REB','STL','TOV','AST','BLK','DREB','FG3A','FG3M',
             'FG3_PCT','FGA','FGM','FG_PCT','FTA','FTM','FT_PCT','MIN',
             'OREB','PF','PLUS_MINUS','VIDEO_AVAILABLE']
df_gamelogs = df_gamelogs[col_order]

# Cumulative totals by team

This number can potentially be computed at the game level or at the season level and reapplied to PER retroactively. 

In [16]:
df_games.loc[df_games.GAME_ID == '0021500391']

,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,SEASON_ID,GAME_DATE,GAME_ID,MATCHUP,WL,PTS,REB,STL,TOV,AST,BLK,DREB,FG3A,FG3M,FG3_PCT,FGA,FGM,FG_PCT,FTA,FTM,FT_PCT,MIN,OREB,PF,PLUS_MINUS,VIDEO_AVAILABLE
0,1610612765,Detroit Pistons,DET,22015,2015-12-18,0021500391,DET @ CHI,W,147,64,7,11,28,4,45,29,8,0.276,124,56,0.452,46,27,0.587,340,19,35,3,1
23,1610612741,Chicago Bulls,CHI,22015,2015-12-18,0021500391,CHI vs. DET,L,144,62,5,17,21,9,42,22,5,0.227,120,50,0.417,44,39,0.886,340,20,35,-3,1


In [17]:
df_games.sort_values('GAME_ID', inplace=True)
df_games.reset_index(drop=True, inplace=True)

In [18]:
df_games = df_games.merge(df_games.groupby(['TEAM_ID', 'GAME_DATE'])['AST', 'FGA', 'FGM', 'PTS'].cumsum(),left_index=True, right_index=True, suffixes=('', '_CUM_TOTALS'))

In [19]:
df_games['GAME_NUMBER'] = df_games.groupby(['TEAM_ID', 'GAME_DATE']).cumcount()+1

In [20]:
df_games.head()

,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,SEASON_ID,GAME_DATE,GAME_ID,MATCHUP,WL,PTS,REB,STL,TOV,AST,BLK,DREB,FG3A,FG3M,FG3_PCT,FGA,FGM,FG_PCT,FTA,FTM,FT_PCT,MIN,OREB,PF,PLUS_MINUS,VIDEO_AVAILABLE,AST_CUM_TOTALS,FGA_CUM_TOTALS,FGM_CUM_TOTALS,PTS_CUM_TOTALS,GAME_NUMBER
0,1610612765,Detroit Pistons,DET,22015,2015-10-27,0021500001,DET @ ATL,W,106,59,5,15,23,3,36,29,12,0.414,96,37,0.385,26,20,0.769,240,23,15,12,1,23,96,37,106,1
1,1610612737,Atlanta Hawks,ATL,22015,2015-10-27,0021500001,ATL vs. DET,L,94,40,9,15,22,4,33,27,8,0.296,82,37,0.451,15,12,0.800,240,7,25,-12,1,22,82,37,94,1
2,1610612739,Cleveland Cavaliers,CLE,22015,2015-10-27,0021500002,CLE @ CHI,L,95,50,5,11,26,7,39,29,9,0.310,94,38,0.404,17,10,0.588,240,11,21,-2,1,26,94,38,95,1
3,1610612741,Chicago Bulls,CHI,22015,2015-10-27,0021500002,CHI vs. CLE,W,97,47,6,13,13,10,40,19,7,0.368,87,37,0.425,23,16,0.696,240,7,22,2,1,13,87,37,97,1
4,1610612740,New Orleans Pelicans,NOP,22015,2015-10-27,0021500003,NOP @ GSW,L,95,33,9,19,21,3,25,18,6,0.333,83,35,0.422,27,19,0.704,240,8,26,-16,1,21,83,35,95,1


In [21]:
df_games = df_games.merge(
    (df_games
        .groupby(['GAME_DATE'])['AST', 'FGA', 'FGM', 'REB', 'OREB', 'TOV', 'FTA', 'FTM', 'PF', 'PTS']
        .sum()),
    left_on='GAME_DATE', right_index=True, 
    suffixes=('', '_LEAGUE'))

In [22]:
g = df_games.groupby('GAME_DATE')['TEAM_ID'].nunique()

In [23]:
g.name = 'NUM_TEAMS'
df_games = df_games.join(g, on='GAME_DATE')

In [24]:
cum_cols = ['AST_LEAGUE', 'FGA_LEAGUE', 'FGM_LEAGUE', 'REB_LEAGUE', 
            'OREB_LEAGUE', 'TOV_LEAGUE', 'FTA_LEAGUE', 'FTM_LEAGUE', 
            'PF_LEAGUE', 'PTS_LEAGUE', 'NUM_TEAMS']

d = df_games.merge(df_games.groupby('GAME_DATE')[cum_cols].cumsum(), 
               left_index=True, right_index=True,
               suffixes = ('', '_CUM'))

In [25]:
d.columns

Index([          u'TEAM_ID',         u'TEAM_NAME', u'TEAM_ABBREVIATION',
               u'SEASON_ID',         u'GAME_DATE',           u'GAME_ID',
                 u'MATCHUP',                u'WL',               u'PTS',
                     u'REB',               u'STL',               u'TOV',
                     u'AST',               u'BLK',              u'DREB',
                    u'FG3A',              u'FG3M',           u'FG3_PCT',
                     u'FGA',               u'FGM',            u'FG_PCT',
                     u'FTA',               u'FTM',            u'FT_PCT',
                     u'MIN',              u'OREB',                u'PF',
              u'PLUS_MINUS',   u'VIDEO_AVAILABLE',    u'AST_CUM_TOTALS',
          u'FGA_CUM_TOTALS',    u'FGM_CUM_TOTALS',    u'PTS_CUM_TOTALS',
             u'GAME_NUMBER',        u'AST_LEAGUE',        u'FGA_LEAGUE',
              u'FGM_LEAGUE',        u'REB_LEAGUE',       u'OREB_LEAGUE',
              u'TOV_LEAGUE',        u'FTA_LEAGUE', 

In [26]:
d.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2192 entries, 0 to 2191
Columns: 56 entries, TEAM_ID to TOV_LEAGUE_CUM
dtypes: float64(3), int64(46), object(7)
memory usage: 976.1+ KB


In [27]:
df_players.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 479 entries, 0 to 478
Columns: 13 entries, DISPLAY_FIRST_LAST to TO_YEAR
dtypes: int64(3), object(10)
memory usage: 52.4+ KB


In [28]:
df_gamelogs.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23753 entries, 0 to 23752
Columns: 28 entries, Player_ID to VIDEO_AVAILABLE
dtypes: float64(3), int64(19), object(6)
memory usage: 5.3+ MB


## Home and Away PPG

In [29]:
d = d.join(d.groupby('GAME_ID')['PTS'].sum(), on='GAME_ID', rsuffix='_GAMETOTAL')

In [30]:
df_gamelogs.columns

Index([         u'Player_ID', u'DISPLAY_FIRST_LAST',          u'SEASON_ID',
                u'GAME_DATE',            u'Game_ID',            u'MATCHUP',
                       u'WL',                u'PTS',                u'REB',
                      u'STL',                u'TOV',                u'AST',
                      u'BLK',               u'DREB',               u'FG3A',
                     u'FG3M',            u'FG3_PCT',                u'FGA',
                      u'FGM',             u'FG_PCT',                u'FTA',
                      u'FTM',             u'FT_PCT',                u'MIN',
                     u'OREB',                 u'PF',         u'PLUS_MINUS',
          u'VIDEO_AVAILABLE'],
      dtype='object')

In [31]:
d.columns

Index([          u'TEAM_ID',         u'TEAM_NAME', u'TEAM_ABBREVIATION',
               u'SEASON_ID',         u'GAME_DATE',           u'GAME_ID',
                 u'MATCHUP',                u'WL',               u'PTS',
                     u'REB',               u'STL',               u'TOV',
                     u'AST',               u'BLK',              u'DREB',
                    u'FG3A',              u'FG3M',           u'FG3_PCT',
                     u'FGA',               u'FGM',            u'FG_PCT',
                     u'FTA',               u'FTM',            u'FT_PCT',
                     u'MIN',              u'OREB',                u'PF',
              u'PLUS_MINUS',   u'VIDEO_AVAILABLE',    u'AST_CUM_TOTALS',
          u'FGA_CUM_TOTALS',    u'FGM_CUM_TOTALS',    u'PTS_CUM_TOTALS',
             u'GAME_NUMBER',        u'AST_LEAGUE',        u'FGA_LEAGUE',
              u'FGM_LEAGUE',        u'REB_LEAGUE',       u'OREB_LEAGUE',
              u'TOV_LEAGUE',        u'FTA_LEAGUE', 

# Computing PER

    uPER = (1 / MP) *
    [ 3P
    + (2/3) * AST
    + (2 - factor * (team_AST / team_FG)) * FG(FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG))) - PF * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) ]
    
    factor = (2 / 3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
    VOP    = lg_PTS / (lg_FGA - lg_ORB + lg_TOV + 0.44 * lg_FTA)
    DRB%   = (lg_TRB - lg_ORB) / lg_TRB
    
Computing PER requires the construction of several league-wide statistics. I'm not sure if the league aggregations are updated as the season progresses or if they are relative to the day the game was played. To this point, I'll compute various possibilities, and use each of them in a separate PER rating.

In [32]:
def compute_FTM_PF(row):
    return row['FTM_LEAGUE_CUM']/row['PF_LEAGUE_CUM']

def compute_FTA_PF(row):
    return row['FTA_LEAGUE_CUM']/row['PF_LEAGUE_CUM']

In [33]:
def compute_factor(row):
    return (2/3) - (.5*row['AST_LEAGUE_CUM']/row['FGM_LEAGUE_CUM']/(2*row['FGM_LEAGUE_CUM']/row['FTM_LEAGUE_CUM']))

In [34]:
def compute_VOP(row):
    return row['PTS_LEAGUE_CUM']/(row['FGA_LEAGUE_CUM']-row['OREB_LEAGUE_CUM']+row['TOV_LEAGUE_CUM']+0.44*row['FTA_LEAGUE_CUM'])

In [35]:
def compute_DRB(row):
    return (row['REB_LEAGUE_CUM']-row['OREB_LEAGUE_CUM'])/row['REB_LEAGUE_CUM']

In [36]:
d['FACTOR'] = d.apply(compute_factor, axis = 1)
d['VOP'] = d.apply(compute_VOP, axis = 1)
d['DRB_PCT'] = d.apply(compute_DRB, axis = 1)
d['FTM_PF'] = d.apply(compute_FTM_PF, axis = 1)
d['FTA_PF'] = d.apply(compute_FTA_PF, axis = 1)

In [37]:
d['PACE_ADJUST'] = (2*d.PTS_LEAGUE_CUM/d.NUM_TEAMS_CUM)/d.PTS_GAMETOTAL

In [38]:
def compute_assisted_FG(row):
    return row['AST_TEAM_DAILY']/row['FGM_TEAM_DAILY']

In [39]:
log_cols = ['Player_ID', 'DISPLAY_FIRST_LAST', 'Game_ID', 'MATCHUP', 'GAME_DATE',
            'MIN', 'FG3M', 'AST', 'FGM', 'FTM',
            'TOV', 'FGA', 'FTA', 'FTM', 'REB', 
            'OREB', 'STL', 'BLK', 'PF']

In [40]:
game_cols = ['GAME_ID', 'MATCHUP','VOP', 'FACTOR', 'DRB_PCT', 'FTM_PF', 'FTA_PF', 'PACE_ADJUST']

In [41]:
df_PER = (df_gamelogs[log_cols]
    .merge(d[game_cols], left_on=['Game_ID', 'MATCHUP'], right_on=['GAME_ID', 'MATCHUP'])
    .merge(d[['GAME_ID', 'MATCHUP', 'AST', 'FGM']], left_on=['Game_ID', 'MATCHUP'], right_on=['GAME_ID', 'MATCHUP'], suffixes=('', '_TEAM_DAILY')))

In [42]:
df_PER.head()

,Player_ID,DISPLAY_FIRST_LAST,Game_ID,GAME_DATE,MIN,FG3M,AST,FGM,FTM,TOV,FGA,FTA,FTM,REB,OREB,STL,BLK,PF,GAME_ID,VOP,FACTOR,DRB_PCT,FTM_PF,FTA_PF,PACE_ADJUST,GAME_ID_TEAM_DAILY,MATCHUP,AST_TEAM_DAILY,FGM_TEAM_DAILY


In [43]:
def line_1(row):
    return 1/row['MIN']

def line_2(row):
    return row['FG3M']

def line_3(row):
    return 2/3*row['AST']

def line_4(row):
    return (2 - row['FACTOR'] * compute_assisted_FG(row)) * row['FGM']

def line_5a(row):
    return row['FTM']*.5

def line_5b(row):
    return 1 + (1 - compute_assisted_FG(row))

def line_5c(row):
    return 2/3*compute_assisted_FG(row)

def line_5(row):
    return line_5a(row)*line_5b(row) + line_5c(row)

def line_6(row):
    return row['VOP']*row['TOV']

def line_7(row):
    return row['VOP']*row['DRB_PCT']*(row['FGA'] - row['FGM'])

def line_8(row):
    return row['VOP']*.44*(.44 + (.56*row['DRB_PCT']))*(row['FTA']-row['FTM'])

def line_9(row):
    return row['VOP']*(1 - row['DRB_PCT'])*(row['REB']-row['OREB'])

def line_10(row):
    return row['VOP']*row['DRB_PCT']*row['OREB']

def line_11(row):
    return row['VOP']*row['STL']

def line_12(row):
    return row['VOP']*row['DRB_PCT']*row['BLK']

def line_13(row):
    return row['PF']*(row['FTM_PF'] - .44*row['FTA_PF']*row['VOP'])

In [44]:
def uPER(row):
    uPER = (line_2(row) + 
        line_3(row) + 
        line_4(row) + 
        line_5(row) -
        line_6(row) -
        line_7(row) -
        line_8(row) +
        line_9(row) +
        line_10(row)+
        line_11(row)+
        line_12(row)-
        line_13(row))
    return uPER[0]

In [45]:
df_PER['uPER'] = df_PER.apply(uPER, axis = 1)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [ ]:
df_PER['aPER'] = df_PER.uPER*df_PER.PACE_ADJUST

In [ ]:
df_PER['PER'] = 0

In [ ]:
df_PER.loc[df_PER.MIN > 0, 'PER'] = df_PER.loc[df_PER.MIN > 0, 'aPER']/df_PER.loc[df_PER.MIN > 0, 'MIN']

In [ ]:
df_PER.head()

In [ ]:
df_PER['GAME_DATE'] = pd.to_datetime(df_PER.GAME_DATE)

In [ ]:
df_min_filter = df_PER.loc[df_PER.MIN >=8].drop(['PER_CUM_SUM', 'PER_CUM_COUNT'], axis=1)

In [ ]:
df_min_filter = (df_min_filter
    .join(df_min_filter.sort_values('GAME_DATE').groupby('GAME_DATE').PER.sum().cumsum(), on='GAME_DATE', rsuffix='_CUM_SUM')
    .join(df_min_filter.sort_values('GAME_DATE').groupby('GAME_DATE').PER.count().cumsum(), on='GAME_DATE', rsuffix='_CUM_COUNT'))

In [ ]:
df_min_filter['AVG_PER'] = df_min_filter['PER_CUM_SUM']/df_min_filter['PER_CUM_COUNT']

In [ ]:
df_min_filter.head()

In [ ]:
df_min_filter['HollingerPER'] = df_min_filter['PER']*(15/df_min_filter['AVG_PER'])

In [ ]:
df_min_filter.loc[(df_min_filter.GAME_DATE == '2015-10-27'), ['DISPLAY_FIRST_LAST', 'HollingerPER']].sort_values('HollingerPER', ascending=False).head(15).reset_index()

In [ ]:
!pwd

In [ ]:
df_min_filter.to_csv('../../../2016-02-Paul/2015_holligerPER.csv')